In [44]:
from pathlib import Path
import json
import re

import torch
from torch import Tensor
import torch.optim as optim
from torch.utils.data import Dataset
from torch import nn
from torch import functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import image as mpl_image
from typing import Type, Any, Callable, Union, List, Optional


In [4]:
batch_size = 4

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5}

In [85]:
test_folder = Path("/nic/test")
model_file = Path("/nic/models/model_20.pyt")
annotations_file = Path("/nic/annotations.json")
sequences_file = Path("/nic/sequences.npy")

In [6]:
[x for x in test_folder.rglob("*.png")]

[WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[101]_frame[0].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[16].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[4].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[6].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[9].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[11].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[1].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[2].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[3].png'),
 WindowsPath('/nic/test/Not_Oriented/video[MAX_210220_NGasl_02

In [107]:
class ImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = path.parts[-2]
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [23]:
class BlindImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = "Unidentified"
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [24]:
def label_to_tensor(label, classes):
    index = classes[label]

    return index

In [25]:
image_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        # torchvision.transforms.Grayscale(num_output_channels=1)
        lambda x: torchvision.transforms.functional.to_pil_image(x),
        lambda x: torchvision.transforms.functional.to_grayscale(x),
        lambda x: torchvision.transforms.functional.to_tensor(x),

    ]
)



target_transform = lambda x: label_to_tensor(x, classes)


In [26]:
testset = BlindImageDataset(test_folder, image_transform, target_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, 
                                        # num_workers=1
                                         drop_last=True
                                        )

# Define the model

In [27]:

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 6,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
#         print(x.shape)
        x = self.layer2(x)
#         print(x.shape)
        x = self.layer3(x)
#         print(x.shape)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [28]:
def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [29]:
def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

# Load the model

In [30]:
model = resnet18()


In [31]:
model.load_state_dict(torch.load(str(model_file)))


<All keys matched successfully>

# Annotate the data with the model

In [98]:
confusion_matrix_test = np.zeros((len(classes), len(classes)))
annotations = {}
true_labels = {}
for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
    path = data["path"]
    inputs = data["image"]
    labels = data["label"]
    
    
    # forward + backward + optimize
    outputs = model(inputs)


    
    for j in [0,1,2,3]:
        confusion_matrix_test[labels[j], torch.argmax(outputs[j])] += 1
        label = int(torch.argmax(outputs[j]).detach().cpu())
        annotations[path[j]] = label
        true_labels[path[j]] = labels[j]
        

# Output the confusion matrix

In [37]:
confusion_matrix_test_table = pd.DataFrame(
data=confusion_matrix_test,
index=list(classes.keys()),
columns=list(classes.keys()))

In [38]:
confusion_matrix_test_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified
Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Slanted,0.0,0.0,0.0,0.0,0.0,0.0
Unidentified,650.0,433.0,692.0,499.0,1071.0,63.0


# Save to json

In [41]:
annotations

{'\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[101]_frame[0].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[16].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[4].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[6].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[102]_frame[9].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[11].png': 2,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[1].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[2].png': 4,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[3].png': 0,
 '\\nic\\test\\Not_Oriented\\video[MAX_210220_NGasl_02_20MLE.tif]_particle[106]_frame[4].png': 0,
 '\\nic\\test\\Not

In [63]:
new_annotations = {}

for path, annotation in annotations.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    print(capture)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_annotations[particle][frame] = annotation
    except:
        new_annotations[particle] = {}
        new_annotations[particle][frame] = annotation

[('101', '0')]
[('102', '16')]
[('102', '4')]
[('102', '6')]
[('102', '9')]
[('106', '11')]
[('106', '1')]
[('106', '2')]
[('106', '3')]
[('106', '4')]
[('106', '8')]
[('106', '9')]
[('108', '0')]
[('109', '10')]
[('109', '13')]
[('109', '2')]
[('109', '3')]
[('109', '4')]
[('109', '5')]
[('109', '8')]
[('109', '9')]
[('110', '10')]
[('110', '12')]
[('110', '15')]
[('110', '17')]
[('110', '18')]
[('110', '2')]
[('110', '6')]
[('110', '7')]
[('110', '8')]
[('110', '9')]
[('113', '0')]
[('113', '10')]
[('113', '11')]
[('113', '12')]
[('113', '13')]
[('113', '15')]
[('113', '16')]
[('113', '17')]
[('113', '18')]
[('113', '19')]
[('113', '2')]
[('113', '7')]
[('113', '9')]
[('114', '12')]
[('114', '13')]
[('114', '16')]
[('114', '19')]
[('114', '2')]
[('116', '10')]
[('116', '11')]
[('116', '1')]
[('116', '2')]
[('116', '3')]
[('116', '4')]
[('116', '5')]
[('116', '6')]
[('116', '7')]
[('116', '9')]
[('117', '10')]
[('117', '11')]
[('117', '19')]
[('117', '5')]
[('117', '6')]
[('117', '7')

[('187', '13')]
[('187', '14')]
[('187', '18')]
[('187', '1')]
[('187', '2')]
[('187', '4')]
[('187', '5')]
[('187', '6')]
[('187', '7')]
[('187', '8')]
[('18', '11')]
[('18', '14')]
[('18', '16')]
[('18', '17')]
[('18', '18')]
[('18', '19')]
[('19', '12')]
[('19', '18')]
[('19', '19')]
[('19', '1')]
[('20', '0')]
[('20', '10')]
[('20', '13')]
[('20', '14')]
[('20', '17')]
[('20', '18')]
[('20', '19')]
[('20', '1')]
[('20', '7')]
[('20', '8')]
[('20', '9')]
[('22', '0')]
[('22', '10')]
[('22', '11')]
[('22', '12')]
[('22', '13')]
[('22', '14')]
[('22', '15')]
[('22', '16')]
[('22', '18')]
[('22', '19')]
[('22', '1')]
[('22', '2')]
[('22', '3')]
[('22', '6')]
[('22', '7')]
[('22', '8')]
[('22', '9')]
[('23', '13')]
[('23', '15')]
[('23', '6')]
[('24', '0')]
[('24', '10')]
[('24', '11')]
[('24', '12')]
[('24', '13')]
[('24', '14')]
[('24', '15')]
[('24', '17')]
[('24', '1')]
[('24', '2')]
[('24', '5')]
[('24', '9')]
[('25', '0')]
[('25', '11')]
[('25', '13')]
[('25', '14')]
[('25', '15')

In [66]:

# for particle in new_annotations:
#     new_annotations[particle] = []
    
#     for j in range(20):
#         new_annotations[particle].append(new_annotations[j])

new_annotations[120]

{10: 0,
 11: 2,
 9: 0,
 0: 3,
 12: 2,
 13: 5,
 15: 2,
 16: 4,
 7: 4,
 18: 4,
 19: 4,
 1: 1,
 17: 4,
 2: 4,
 3: 3,
 4: 4,
 5: 4,
 6: 4,
 8: 4}

In [89]:
sequences = {}

k = 0
sequence = []
for particle in new_annotations:
#     print(particle)
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_annotations[particle][j]
            sequence.append(next_annotation)
        except:
#             sequences[k] = sequence
#             k = k + 1
#             sequence = []
            sequence.append(6)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [90]:
sequences

{0: [2, 4, 1, 4, 4, 3, 3, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1],
 1: [0, 2, 0, 4, 0, 2, 0, 4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4],
 2: [0, 0, 4, 0, 0, 4, 4, 4, 2, 4, 4, 2, 4, 5, 4, 4, 2, 4, 2, 4],
 3: [0, 4, 4, 4, 4, 3, 4, 1, 4, 2, 4, 4, 4, 2, 4, 3, 4, 2, 5, 2],
 4: [0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 2, 2, 0, 2, 2, 2],
 5: [2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 4],
 6: [0, 0, 0, 0, 4, 2, 2, 0, 4, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0],
 7: [2, 2, 0, 2, 2, 4, 4, 4, 4, 4, 2, 2, 0, 0, 2, 4, 0, 4, 2, 2],
 8: [0, 4, 4, 2, 0, 2, 2, 0, 4, 0, 2, 0, 0, 0, 0, 4, 0, 2, 4, 2],
 9: [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 4, 2, 0],
 10: [0, 2, 0, 2, 0, 0, 2, 2, 2, 4, 0, 2, 4, 0, 0, 2, 4, 0, 2, 2],
 11: [3, 1, 4, 3, 4, 4, 4, 4, 4, 0, 0, 2, 2, 5, 6, 2, 4, 4, 4, 4],
 12: [0, 2, 4, 4, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 4, 4, 2, 5],
 13: [0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 4, 4, 2, 2, 2, 2],
 14: [4, 0, 0, 2, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
 15: 

In [91]:
len(sequences)

172

In [92]:
[len(sequence) for sequence in sequences.values()]

[20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20]

In [80]:
full_sequences = [sequence for sequence in sequences.values() if len(sequence) == 20]

In [96]:
sequences_array = np.array(list(sequences.values()))
sequences_array.shape

(172, 20)

In [97]:
np.save(str(sequences_file), sequences_array)

In [102]:
new_true_labels = {}

for path, annotation in true_labels.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    print(capture)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_true_labels[particle][frame] = annotation
    except:
        new_true_labels[particle] = {}
        new_true_labels[particle][frame] = annotation

[('101', '0')]
[('102', '16')]
[('102', '4')]
[('102', '6')]
[('102', '9')]
[('106', '11')]
[('106', '1')]
[('106', '2')]
[('106', '3')]
[('106', '4')]
[('106', '8')]
[('106', '9')]
[('108', '0')]
[('109', '10')]
[('109', '13')]
[('109', '2')]
[('109', '3')]
[('109', '4')]
[('109', '5')]
[('109', '8')]
[('109', '9')]
[('110', '10')]
[('110', '12')]
[('110', '15')]
[('110', '17')]
[('110', '18')]
[('110', '2')]
[('110', '6')]
[('110', '7')]
[('110', '8')]
[('110', '9')]
[('113', '0')]
[('113', '10')]
[('113', '11')]
[('113', '12')]
[('113', '13')]
[('113', '15')]
[('113', '16')]
[('113', '17')]
[('113', '18')]
[('113', '19')]
[('113', '2')]
[('113', '7')]
[('113', '9')]
[('114', '12')]
[('114', '13')]
[('114', '16')]
[('114', '19')]
[('114', '2')]
[('116', '10')]
[('116', '11')]
[('116', '1')]
[('116', '2')]
[('116', '3')]
[('116', '4')]
[('116', '5')]
[('116', '6')]
[('116', '7')]
[('116', '9')]
[('117', '10')]
[('117', '11')]
[('117', '19')]
[('117', '5')]
[('117', '6')]
[('117', '7')

[('52', '16')]
[('52', '1')]
[('52', '2')]
[('52', '3')]
[('52', '4')]
[('52', '6')]
[('52', '8')]
[('52', '9')]
[('53', '12')]
[('53', '15')]
[('53', '1')]
[('53', '2')]
[('53', '4')]
[('53', '5')]
[('53', '6')]
[('53', '7')]
[('53', '9')]
[('54', '0')]
[('54', '16')]
[('54', '8')]
[('54', '9')]
[('55', '12')]
[('55', '14')]
[('55', '17')]
[('55', '3')]
[('55', '6')]
[('56', '11')]
[('56', '12')]
[('56', '14')]
[('56', '16')]
[('56', '17')]
[('56', '1')]
[('56', '3')]
[('56', '4')]
[('56', '5')]
[('56', '7')]
[('56', '8')]
[('56', '9')]
[('57', '4')]
[('58', '10')]
[('58', '12')]
[('58', '13')]
[('58', '14')]
[('58', '16')]
[('58', '17')]
[('58', '18')]
[('58', '19')]
[('58', '3')]
[('58', '4')]
[('58', '6')]
[('58', '7')]
[('58', '8')]
[('58', '9')]
[('59', '0')]
[('59', '10')]
[('59', '11')]
[('59', '12')]
[('59', '13')]
[('59', '14')]
[('59', '15')]
[('59', '16')]
[('59', '18')]
[('59', '2')]
[('59', '4')]
[('59', '5')]
[('59', '6')]
[('59', '7')]
[('59', '8')]
[('59', '9')]
[('5',

In [105]:
sequences = {}

k = 0
sequence = []
for particle in new_true_labels:
#     print(particle)
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_true_labels[particle][j]
            sequence.append(next_annotation.item())
        except:
#             sequences[k] = sequence
#             k = k + 1
#             sequence = []
            sequence.append(6)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [106]:
sequences

{0: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 1: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 2: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 3: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 4: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 5: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 6: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 7: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 8: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 9: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 10: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 11: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5],
 12: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 13: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 14: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 15: 

In [108]:
testset = ImageDataset(test_folder, image_transform, target_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, 
                                        # num_workers=1
                                         drop_last=True
                                        )

In [109]:
confusion_matrix_test = np.zeros((len(classes), len(classes)))
annotations = {}
true_labels = {}
for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
    path = data["path"]
    inputs = data["image"]
    labels = data["label"]
    
    
    # forward + backward + optimize
    outputs = model(inputs)


    
    for j in [0,1,2,3]:
        confusion_matrix_test[labels[j], torch.argmax(outputs[j])] += 1
        label = int(torch.argmax(outputs[j]).detach().cpu())
        annotations[path[j]] = label
        true_labels[path[j]] = labels[j]
        

In [110]:
new_true_labels = {}

for path, annotation in true_labels.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    print(capture)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_true_labels[particle][frame] = annotation
    except:
        new_true_labels[particle] = {}
        new_true_labels[particle][frame] = annotation

[('101', '0')]
[('102', '16')]
[('102', '4')]
[('102', '6')]
[('102', '9')]
[('106', '11')]
[('106', '1')]
[('106', '2')]
[('106', '3')]
[('106', '4')]
[('106', '8')]
[('106', '9')]
[('108', '0')]
[('109', '10')]
[('109', '13')]
[('109', '2')]
[('109', '3')]
[('109', '4')]
[('109', '5')]
[('109', '8')]
[('109', '9')]
[('110', '10')]
[('110', '12')]
[('110', '15')]
[('110', '17')]
[('110', '18')]
[('110', '2')]
[('110', '6')]
[('110', '7')]
[('110', '8')]
[('110', '9')]
[('113', '0')]
[('113', '10')]
[('113', '11')]
[('113', '12')]
[('113', '13')]
[('113', '15')]
[('113', '16')]
[('113', '17')]
[('113', '18')]
[('113', '19')]
[('113', '2')]
[('113', '7')]
[('113', '9')]
[('114', '12')]
[('114', '13')]
[('114', '16')]
[('114', '19')]
[('114', '2')]
[('116', '10')]
[('116', '11')]
[('116', '1')]
[('116', '2')]
[('116', '3')]
[('116', '4')]
[('116', '5')]
[('116', '6')]
[('116', '7')]
[('116', '9')]
[('117', '10')]
[('117', '11')]
[('117', '19')]
[('117', '5')]
[('117', '6')]
[('117', '7')

[('95', '19')]
[('95', '9')]
[('96', '10')]
[('96', '14')]
[('96', '15')]
[('96', '18')]
[('96', '3')]
[('96', '7')]
[('97', '0')]
[('97', '13')]
[('97', '14')]
[('97', '15')]
[('97', '17')]
[('97', '18')]
[('97', '19')]
[('97', '1')]
[('97', '2')]
[('97', '3')]
[('97', '6')]
[('97', '7')]
[('97', '8')]
[('99', '1')]
[('100', '10')]
[('100', '12')]
[('100', '14')]
[('100', '17')]
[('100', '18')]
[('100', '19')]
[('100', '1')]
[('100', '2')]
[('100', '4')]
[('100', '6')]
[('100', '8')]
[('101', '10')]
[('101', '1')]
[('101', '4')]
[('101', '5')]
[('101', '8')]
[('102', '15')]
[('102', '2')]
[('103', '10')]
[('103', '12')]
[('103', '19')]
[('103', '7')]
[('103', '8')]
[('103', '9')]
[('105', '16')]
[('105', '18')]
[('105', '8')]
[('106', '0')]
[('106', '10')]
[('106', '12')]
[('106', '13')]
[('106', '14')]
[('106', '15')]
[('106', '16')]
[('106', '17')]
[('106', '18')]
[('106', '19')]
[('106', '5')]
[('106', '6')]
[('106', '7')]
[('108', '10')]
[('108', '11')]
[('108', '12')]
[('108', '1

In [111]:
sequences = {}

k = 0
sequence = []
for particle in new_true_labels:
#     print(particle)
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_true_labels[particle][j]
            sequence.append(next_annotation.item())
        except:
#             sequences[k] = sequence
#             k = k + 1
#             sequence = []
            sequence.append(6)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [112]:
sequences

{0: [0, 4, 1, 1, 4, 4, 3, 1, 4, 3, 4, 3, 3, 3, 1, 3, 3, 3, 3, 3],
 1: [2, 2, 4, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 4, 0, 2, 2, 2],
 2: [4, 0, 0, 0, 0, 4, 4, 4, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4],
 3: [0, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 2, 4],
 4: [2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2],
 5: [2, 2, 0, 2, 4, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 4, 0, 0, 2],
 6: [0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
 7: [2, 2, 0, 2, 2, 2, 2, 4, 4, 4, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0],
 8: [2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 2, 2, 2, 2],
 9: [2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0],
 10: [0, 2, 0, 0, 0, 0, 0, 0, 2, 4, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2],
 11: [1, 3, 4, 4, 4, 4, 4, 2, 4, 0, 0, 0, 2, 2, 6, 2, 2, 4, 3, 3],
 12: [0, 2, 4, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 4, 2, 2, 4],
 13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 4, 4, 4],
 14: [2, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 15: 

In [146]:
naive_transition_matrix = np.zeros((7,7))
for sequence in sequences.values():
    prev_state = 6
    for state in sequence:
#         print(state)
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state
naive_transition_matrix

array([[366.,   0., 174.,   1.,  42.,   6.,   1.],
       [  0., 128.,   2., 112.,  50.,   1.,   0.],
       [144.,   4., 394.,  16., 166.,  16.,   5.],
       [  0.,  89.,  19., 290., 133.,  14.,   3.],
       [ 54.,  45., 153., 132., 506.,  28.,   3.],
       [  3.,   4.,  17.,   9.,  22.,  88.,   4.],
       [ 38.,  30.,  33.,  20.,  51.,   8.,  16.]])

In [147]:
sums = np.sum(naive_transition_matrix, axis=1)
sums

array([590., 293., 745., 548., 921., 147., 196.])

In [148]:

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
naive_transition_matrix = naive_transition_matrix / sums.reshape((-1,1))
print(np.sum(naive_transition_matrix, axis=1))

confusion_matrix_test_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table 

[1. 1. 1. 1. 1. 1. 1.]


,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,0.620339,0.000000,0.294915,0.001695,0.071186,0.010169,0.001695
Oriented,0.000000,0.436860,0.006826,0.382253,0.170648,0.003413,0.000000
Precieved_Not_Oriented,0.193289,0.005369,0.528859,0.021477,0.222819,0.021477,0.006711
Precieved_Oriented,0.000000,0.162409,0.034672,0.529197,0.242701,0.025547,0.005474
Slanted,0.058632,0.048860,0.166124,0.143322,0.549403,0.030402,0.003257
Unidentified,0.020408,0.027211,0.115646,0.061224,0.149660,0.598639,0.027211
No_sample,0.193878,0.153061,0.168367,0.102041,0.260204,0.040816,0.081633


In [150]:
transition_matrix = naive_transition_matrix
print(f"Annotated transitions")
print(f"Oriented to slanted: {transition_matrix[1, 4] + transition_matrix[3,4]}")
print(f"slanted to oriented: {transition_matrix[4, 1] + transition_matrix[4,3]}")
print(f"unoriented to slanted: {transition_matrix[0, 4] + transition_matrix[2,4]}")
print(f"slanted to unoriented: {transition_matrix[4, 0] + transition_matrix[4,2]}")


Annotated transitions
Oriented to slanted: 0.4133491940908298
slanted to oriented: 0.19218241042345277
unoriented to slanted: 0.2940052326242748
slanted to unoriented: 0.2247557003257329
